In [ ]:
!pip install langchain_openai==0.3.0
!pip install langchain_community==0.3.14
!pip install chromadb==0.5.0
!pip install sentence_transformers==3.3.1
!pip install langchain_huggingface==0.1.2
!pip install langchain_chroma==0.2.0
!pip install langchain_core==0.3.29
!pip install pydantic==2.10.5
!pip install onnxruntime==1.20.1

In [ ]:
import os
import random
import torch
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
FILE_PATH = "prikaz-po-VKR-bak-i-mag_izmeneniya.txt"
CHUNK_SIZE = 1_000
CHUNK_OVERLAP = 100
N_RANDOM = 3

embedder = HuggingFaceEmbeddings(model_name=MODEL_NAME)

loader = TextLoader(FILE_PATH, encoding="windows-1251")
docs = loader.load()

# Разбиение на фрагменты
splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
chunks = splitter.split_documents(docs)

print(f"Всего фрагментов: {len(chunks)}")

# Случайные фрагменты
sampled = random.sample(chunks, min(N_RANDOM, len(chunks)))
for i, ch in enumerate(sampled, 1):
    preview = ch.page_content.strip()
    if len(preview) > 400:
        preview = preview[:400] + " …"
    print(f"Фрагмент {i}\n{preview}")

# Эмбеддинги
vecs = embedder.embed_documents([c.page_content for c in sampled])
tensor = torch.tensor(vecs)
print("\nРазмерность тензора:", tensor.shape)

Всего фрагментов: 164
Фрагмент 1
1. Изложить пункт 1.6 Положения в следующей редакции:
     «1.6. ВКР может быть подготовлена и представлена к защите в виде реализации стартап-проекта (далее — ВКР-стартап), который представляет собой формализованный отчет об основных целях, задачах, положениях и результатах реализуемого (или планируемого к реализации) бизнес-проекта, разработанного обучающимся или несколькими обучающимися, демонс …
Фрагмент 2
6.3. Для рассмотрения апелляции секретарь ГЭК направляет в апелляционную комиссию по электронной почте копию протокола заседания ГЭК, заключение председателя ГЭК о соблюдении процедурных вопросов при проведении аттестационного испытания, сканы (фото) иных документов, подготовленных членами ГЭК в процессе аттестационного испытания. В случае апелляции по проведению защиты ВКР дополнительно направляе …
Фрагмент 3
Приложение № 2 к приказу Финуниверситета от
Приложение УТВЕРЖДЕН приказом Финуниверситета от 15.10.2020 № 1838/0
                       РЕГ

In [ ]:
from langchain_community.vectorstores import Chroma

vectordb = Chroma.from_documents(documents=chunks, embedding=embedder, collection_name="vkr_docs", persist_directory="tmp")
vectordb.persist()

retriever = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 3}) # mmr убирает максимально похожие записи
QUERY = str(input())
relevant_docs = retriever.invoke(QUERY)

print(f"3 релевантных ответа на запрос: {QUERY}:")
for i, doc in enumerate(relevant_docs, 1):
    text = doc.page_content.strip()
    if len(text) > 400:
        text = text[:400] + " …"
    print(f"Фрагмент {i}\n{text}")

Сколько времени отводится на доклад?
3 релевантных ответа на запрос: Сколько времени отводится на доклад?:
Фрагмент 1
4.6. При проведении защиты ВКР обучающийся по программе бакалавриата выступает с докладом и презентацией в течение 7-10 минут (не более 15 минут для обучающихся по программам магистратуры). Конкретное максимальное время для доклада по ВКР озвучивается председателем ГЭК перед началом работы ГЭК.
     По окончании доклада члены ГЭК задают обучающемуся вопросы по ВКР и его докладу. При ответах на воп …
Фрагмент 2
В заключительной части Доклада характеризуется значимость полученных результатов и Даются общие выводы.
На доклад обучающемуся отводится не более 10 минут. Для программ магистратуры Доклад Должен включать в себя: обоснование актуальности избранной темы; описание научной проблемы и формулировку цели работы; положения, ВЫНОСИМЫе на защиту; практическую значимость работы.
     В заключительной части  …
Фрагмент 3
2.9. В исключительных случаях изменение темы ВКР возмо

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain, StuffDocumentsChain
from google.colab import userdata

TEMPLATE = """Ответь на вопрос: {question}

Вот несколько документов, которые могут помочь:
{context}

Дай ответ, основанный только на предоставленных документах. Если ответ не найден в документах, ответьте "я не уверен"."""

QA_PROMPT = ChatPromptTemplate.from_messages([("human", TEMPLATE)])

model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get('OPENROUTER_KEY'),
    model_name="mistralai/mistral-7b-instruct:free",
)

combine_chain = LLMChain(llm=model, prompt=QA_PROMPT)

stuff_chain = StuffDocumentsChain(llm_chain=combine_chain, document_variable_name="context")

qa_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    return_source_documents=True,
)

QUESTION = "Сколько времени отводится на доклад?"
result = qa_chain.invoke({"query": QUESTION})

print(result["result"])
for doc in result["source_documents"]:
    snippet = doc.page_content.strip().replace("\n", " ")
    print(f"- {snippet[:120]}…")

 Из предоставленных документов следует, что на доклад обучающегося отводится не более 10 минут для бакалавриата и 15 минут для магистратуры. В этих докладах обучающийся должна представлять свою работу и полученные результаты, обсуждать актуальность темы, описать научную проблему и формулировку цели работы, а также объявлять общее заключение и интересные результаты. Затем происходит взаимодействие с членами ГЭК, где обучающийся отвечает на их вопросы. В докладе также рекомендуется использовать презентацию.
- 4.6. При проведении защиты ВКР обучающийся по программе бакалавриата выступает с докладом и презентацией в течение 7-10 …
- В заключительной части Доклада характеризуется значимость полученных результатов и Даются общие выводы. На доклад обучаю…
- 2.9. В исключительных случаях изменение темы ВКР возможно не позднее, чем за 1 месяц, а уточнение темы — не позднее, чем…


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from google.colab import userdata

model = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=userdata.get("OPENROUTER_KEY"),
    model_name="mistralai/mistral-7b-instruct:free",
)

TEMPLATE = """Ответь на вопрос: {question}

Вот несколько документов, которые могут помочь:
{context}

Дай ответ, основанный только на предоставленных документах. Если ответ не найден в документах, ответьте "я не уверен"."""


QA_PROMPT = ChatPromptTemplate.from_messages([("human", TEMPLATE)])

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conv_chain = ConversationalRetrievalChain.from_llm(
    llm=model,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT},
    return_source_documents=False,
    output_key="answer",
)

print('Введите вопрос (или "exit" для выхода):')
while True:
    user_input = input("You: ").strip()
    if user_input.lower() == "exit":
        break
    result = conv_chain.invoke({"question": user_input})
    print("\nAssistant:", result["answer"], "\n")

Введите вопрос (или "exit" для выхода):
You: Что такое ВКР?

Assistant:  ВКР (Вопрос-ответная разработка, также известная как вид работа или дипломная работа) - это самостоятельное исследование, выполняемое студентом по программе бакалавриата или магистратуры, в котором анализируется теоретическая или практическая проблема в области профессиональной деятельности, с соответствующим уровнем подготовленности к самостоятельной профессиональной деятельности. ВКР демонстрирует подготовленность студента, выполняет исследовательское характер, содержит научные обобщения, практические рекомендации и имеет уверяние для публичной защиты. Все эти вопросы, а также оценка ВКР с помощью «Антиплагиат.ВУЗ», указываются в предоставленных документах. 

You: Когда должны быть доведены темы ВКР?

Assistant:  Исходя из предоставленных документов, определить строгий срок для выбора и согласования темы дипломной работы или курсовой не представляется возможным, поскольку дата издания приказа о закреплении темы 